In [1]:
import pandas as pd
import sys
sys.path.append(r"/Users/wrngnfreeman/Github/Shelter-Animal-Outcomes-by-kaggle.com/src")
import data_processing

In [3]:
train_df = data_processing.process_data(
    file_path=r"/Users/wrngnfreeman/Library/CloudStorage/OneDrive-Personal/shared_projects/Shelter Animal Outcomes/raw_data/train.csv",
    AnimalID=r"AnimalID"
)
display(train_df)

,AnimalID,OutcomeType,Name,DateTime,SexuponOutcome,Sterilization,Mix,OutcomeCode,AnimalType_Cat,Sex_Female,...,CoatColor_White Lynx,CoatColor_Yellow,CoatColor_Yellow,CoatPattern_Brindle,CoatPattern_Merle,CoatPattern_Point,CoatPattern_Smoke,CoatPattern_Tabby,CoatPattern_Tick,CoatPattern_Tiger
0,A671945,Return_to_owner,Hambone,2014-02-12 18:22:00,Male,Neutered,Mix,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1,A671945,Return_to_owner,Hambone,2014-02-12 18:22:00,Male,Neutered,Mix,2,0,0,...,0,0,0,0,0,0,0,0,0,0
2,A656520,Euthanasia,Emily,2013-10-13 12:44:00,Female,Spayed,Mix,5,1,1,...,0,0,0,0,0,0,0,1,0,0
3,A686464,Adoption,Pearce,2015-01-31 12:28:00,Male,Neutered,Mix,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,A686464,Adoption,Pearce,2015-01-31 12:28:00,Male,Neutered,Mix,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45725,A698128,Adoption,Zeus,2015-03-09 13:33:00,Male,Neutered,Mix,1,0,0,...,0,0,0,0,0,0,0,0,0,0
45726,A698128,Adoption,Zeus,2015-03-09 13:33:00,Male,Neutered,Mix,1,0,0,...,0,0,0,0,0,0,0,0,0,0
45727,A677478,Transfer,NaN,2014-04-27 12:22:00,Male,Intact,Mix,3,1,0,...,0,0,0,0,0,0,0,0,0,0
45728,A706629,Transfer,NaN,2015-07-02 09:00:00,Male,Intact,Mix,3,1,0,...,0,0,0,0,0,0,0,1,0,0


In [2]:
test_df = data_processing.process_data(
    file_path=r"/Users/wrngnfreeman/Library/CloudStorage/OneDrive-Personal/shared_projects/Shelter Animal Outcomes/raw_data/test.csv",
    AnimalID=r"ID"
)
display(test_df)

,ID,Name,DateTime,SexuponOutcome,Sterilization,Mix,AnimalType_Cat,Sex_Female,Age_15+ years,Age_<1 month,...,CoatColor_White Lynx,CoatColor_Yellow,CoatColor_Yellow,CoatPattern_Brindle,CoatPattern_Merle,CoatPattern_Point,CoatPattern_Smoke,CoatPattern_Tabby,CoatPattern_Tick,CoatPattern_Tiger
0,1,Summer,2015-10-12 12:15:00,Female,Intact,Mix,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,Summer,2015-10-12 12:15:00,Female,Intact,Mix,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,Cheyenne,2014-07-26 17:59:00,Female,Spayed,Mix,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2,Cheyenne,2014-07-26 17:59:00,Female,Spayed,Mix,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2,Cheyenne,2014-07-26 17:59:00,Female,Spayed,Mix,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19562,11453,NaN,2014-10-21 12:57:00,Female,Intact,Mix,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
19563,11454,NaN,2014-09-29 09:00:00,Female,Intact,Mix,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
19564,11455,Rambo,2015-09-05 17:16:00,Male,Neutered,Mix,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19565,11455,Rambo,2015-09-05 17:16:00,Male,Neutered,Mix,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
